# Pair Comparison Models: Another Practical Exercise

## Bradley-Terry model and PageRank

As used in the today’s first practical: The citations dataset in R package BradleyTerryScalable (a CRAN package published from a mini-project by an OxWaSP student, Ella Kaye) gives citation counts for a small network of statistics journals.

In this exercise we establish the relationship between the Bradley-Terry model and a scaled version of (un-damped) PageRank.

The scaling will convert the PageRank scores — i.e., the equilibrium vector of a Markov chain that moves from journal to journal by following cited references — into a vector that contains PageRank score per outgoing citation. (David Selby, a PhD student at Warwick, has invented the nice name “ScroogeFactor” for the resultant vector — because it is like PageRank (a.k.a. “EigenFactor” when applied to inter-journal citation data) but it explicitly rewards those journals that themselves make fewer citations.)

We will see that the Scrooge vector is an estimator for the Bradley-Terry model — i.e., is an alternative to the MLE. With real data, the Scrooge vector will usually be close to the Bradley-Terry model MLE. And when the input matrix has exact Bradley-Terry structure, the ScroogeFactor delivers exactly the Bradley-Terry “ability” scores. (This is a kind of consistency, sometimes called Fisher consistency.)

## Compute the MLE for the Bradley-Terry model

The values of citations_mle $ pi are scaled (arbitrarily) to have mean 1. You should check for yourself how the estimated ability values computed here relate exactly (up to numerical rounding) to the estimated log-ability values that were found by using glm() in the previous practical.

In [18]:
install.packages('BradleyTerryScalable')

Installing package into ‘/homes/thornton/R/x86_64-pc-linux-gnu-library/3.5’
(as ‘lib’ is unspecified)


In [2]:
library(BradleyTerryScalable) 

In [9]:
?citations

In [3]:
citations_mle <- btfit(btdata(citations), a = 1)   ## compute the MLE
citations_mle $ pi                                 ## extract the Bradley-Terry "ability" scores

$full_dataset
      JRSS-B   Biometrika         JASA Comm Statist 
  1.75839650   1.34144779   0.82990805   0.07024766

## Compute the scaled PageRank, or “Scrooge” vector

Now let’s compare that maximum-likelihood fit of the Bradley-Terry model with the (suitably scaled) solution vector from the (un-damped) PageRank algorithm.

We will make a function to compute the “ScroogeFactor” for any given input matrix:

In [13]:
scrooge <- function(m){
  
  ## Matrix m has its rows indexed by the "cited" items, 
  ## and columns by "citing" items
  
  ## First convert m into a transition matrix, with column sums 1
  column_totals <- colSums(m)
  m_transition <- m / matrix(column_totals, 4, 4, byrow = TRUE)
  m_transition <- t(m_transition)
  ## Now we get the eigenvector that corresponds to eigenvalue 1 
  ## (the first eigenvalue, so we want the first column from the matrix of eigenvectors):
  pagerank_vector <- (eigen(m_transition) $ vectors)[, 1]
  
  ## And then _scale_ the pagerank vector by the journals' out-citation counts, 
  ## and then (arbitrarily) re-normalize to have mean 1:
  pagerank_scaled <- pagerank_vector / column_totals
  pagerank_scaled <- pagerank_scaled / mean(pagerank_scaled)
  return(pagerank_scaled)
}

In [14]:
scrooge(citations)

Biometrika Comm Statist         JASA       JRSS-B 
   0.8567506    0.5158066    0.5896434    2.0377994

In [15]:
citations_BT_fitted <- as.matrix(fitted(citations_mle))
scrooge(citations_BT_fitted)

JRSS-B   Biometrika         JASA Comm Statist 
   2.0387097    0.8563767    0.5893085    0.5156051